In [1]:
#import mysql,time and threading
import mysql.connector
import time
import threading

In [4]:
# Function to insert account details into the Mysql database
def insert_account_details(account_name, phone_number, pin, balance):
    try:
        # Establish a connection to the MySQL database
        db = mysql.connector.connect(
            host="localhost",
            user="root",
            password="Siddhu@5",
            database="atm_db"
        )
        
        cursor = db.cursor()

        insert_query = "INSERT INTO accounts (account_name,phone_number,pin,balance) VALUES (%s, %s, %s, %s)"

        values = (account_name, phone_number, pin, balance)

        cursor.execute(insert_query, values)

        # Commit the changes to the database
        db.commit()

        # Close the cursor and database connection
        cursor.close()
        db.close()

        print("Account details inserted successfully.")
    
    except mysql.connector.Error as err:
        print("Error:", err)
        # Handle the error (e.g., log it, provide feedback to the user)

# Example usage:
if __name__ == "__main__":
    account_name = "K.Durga"
    phone_number="9056984576"
    pin = "1234"
    balance = "510342.00"

    insert_account_details(account_name, phone_number, pin, balance)


Account details inserted successfully.


In [2]:
# Connect to MySQL
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Siddhu@5",
    database="atm_db"
)

cursor = db.cursor()

def authenticate(account_number, pin):
    query = "SELECT * FROM accounts WHERE account_number = %s AND pin = %s"
    cursor.execute(query, (account_number, pin))
    return cursor.fetchone() is not None

def check_balance(account_number):
    query = "SELECT balance FROM accounts WHERE account_number = %s"
    cursor.execute(query, (account_number,))
    return cursor.fetchone()[0]

def withdraw_money(account_number, amount):
    current_balance = check_balance(account_number)
    if amount > current_balance:
        return False
    new_balance = current_balance - amount
    update_query = "UPDATE accounts SET balance = %s WHERE account_number = %s"
    cursor.execute(update_query, (new_balance, account_number))
    db.commit()
    return True

def deposit_money(account_number, amount):
    current_balance = check_balance(account_number)
    new_balance = current_balance + amount
    update_query = "UPDATE accounts SET balance = %s WHERE account_number = %s"
    cursor.execute(update_query, (new_balance, account_number))
    db.commit()

def record_transaction(account_number, transaction_type, amount):
    insert_query = "INSERT INTO transactions (account_number, transaction_type, amount) VALUES (%s, %s, %s)"
    cursor.execute(insert_query, (account_number, transaction_type, amount))
    db.commit()

In [ ]:
#handle user input with a specified timeout
def get_input_with_timeout(prompt, timeout):
    result = [None]
    
    def input_thread():
        result[0] = input(prompt)
    
    thread = threading.Thread(target=input_thread)
    thread.daemon = True
    thread.start()
    thread.join(timeout)
    
    if thread.is_alive():
        print("Input timeout \"Enter input value less than 10sec\" Please try again.")
        return None
    else:
        return result[0]
    
def welcome():
    print("""
-------------------------------------------------
        WELCOME TO SIDDI VINAYAKA ATM
-------------------------------------------------
        
         "PLEASE INSERT YOUR CARD"  """)  

def display():
    print("""================================================
             Atm project
================================================
             1. Check Balance
             2. Withdraw Money
             3. Deposit Money
             4. Exit
=================================================""")

def run_atm():
    welcome()
    account_number = get_input_with_timeout("Enter Account Number: ", 10)  # Set a 10-second input timeout
    if account_number is None:
        return

    pin = get_input_with_timeout("Enter PIN: ", 10)  # Set a 10-second input timeout
    if pin is None:
        return

    if authenticate(account_number, pin):
        print("Authentication successful.")

        while True:
            display()
            option = get_input_with_timeout("Select an option (1-4): ", 10)  # Set a 10-second input timeout
            if option is None:
                break

            if option == "1":
                try:
                    balance = check_balance(account_number)
                    print("Your Balance: $", balance)
                except mysql.connector.Error as err:
                    print("Database error:", err)

            elif option == "2":
                try:
                    amount = int(get_input_with_timeout("Enter amount to withdraw: ", 10))  # Set a 10-second input timeout
                    if amount is None:
                        continue
                    
                    if withdraw_money(account_number, amount):
                        print("Withdrawal successful. Remaining balance: $", check_balance(account_number))
                        record_transaction(account_number, "Withdrawal", amount)
                    else:
                        print("Insufficient funds.")
                except ValueError:
                    print("Invalid input. Please enter a valid amount.")
                except mysql.connector.Error as err:
                    print("Database error:", err)

            elif option == "3":
                try:
                    amount = int(get_input_with_timeout("Enter amount to deposit: ", 10))  # Set a 10-second input timeout
                    if amount is None:
                        continue
                    
                    deposit_money(account_number, amount)
                    print("Deposit successful. Updated balance: $", check_balance(account_number))
                    record_transaction(account_number, "Deposit", amount)
                except ValueError:
                    print("Invalid input. Please enter a valid amount.")
                except mysql.connector.Error as err:
                    print("Database error:", err)

            elif option == "4":
                print("Thank you for using the ATM. Goodbye!")
                break
            else:
                print("Invalid option. Try again.")
    else:
        print("Authentication failed \"YOU ENTERED INCORRECT PIN\" Please try again.")

        # Log the failed authentication attempt with timestamp
        with open("authentication_log.txt", "a") as log_file:
            log_file.write(f"Failed authentication attempt for account {account_number} at {time.ctime()}\n")

run_atm()


-------------------------------------------------
        WELCOME TO SIDDI VINAYAKA ATM
-------------------------------------------------
        
         "PLEASE INSERT YOUR CARD"  
